In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv("hospital_data_sampleee.csv")
df.info()

In [ ]:
df["Check-in Time"] = pd.to_datetime(df["Check-in Time"]) - pd.Timestamp('today').normalize()
df["Entry Time"] = pd.to_datetime(df["Entry Time"]) - pd.Timestamp('today').normalize()
df["Completion Time"] = pd.to_datetime(df["Completion Time"]) - pd.Timestamp('today').normalize()
df.describe()

In [ ]:
df['Waiting Time'] = (df['Entry Time'] - df['Check-in Time'])  / pd.Timedelta('1 minute')
df['Check-in Time'] = df['Check-in Time'] / pd.Timedelta('1 hour')
df['Entry Time'] = df['Entry Time'] / pd.Timedelta('1 hour')
df['Completion Time'] = df['Completion Time'] / pd.Timedelta('1 hour')
df.describe()

In [ ]:
df

In [ ]:
df.plot.scatter(x='Entry Time', y='Completion Time')

In [ ]:
# Remove wrong data
df = df.loc[df['Entry Time'] <= df['Completion Time']]
df.plot.scatter(x='Entry Time', y='Completion Time')

In [ ]:
df.plot.scatter(x='Check-in Time', y='Waiting Time')

In [ ]:
df = df.loc[df['HospitalID'] == 'HOSP1']
df = df.drop(columns=['Date', 'HospitalID'])
df

In [ ]:
PATIENT_IN_ROOM = []
for i in range(80, 240):
     t = i / 10
     PATIENT_IN_ROOM.append(len(df.loc[df['Check-in Time'] <= t].loc[t < df['Entry Time']]))
plt.xlabel('Time')
plt.ylabel('Patients in line')
plt.plot(np.array(range(80, 240))/10 ,PATIENT_IN_ROOM)

In [ ]:
df['Entry Time'] = df['Entry Time'].round(decimals=1)
df['Check-in Time'] = df['Check-in Time'].round(decimals=1)
df['Completion Time'] = df['Completion Time'].round(decimals=1)
df = df.groupby('Entry Time').mean().reset_index()
df

In [ ]:
pdf = pd.DataFrame({ 'Entry Time': np.array(range(80, 240)) / 10, 'Patient in line': PATIENT_IN_ROOM })
df = df.join(pdf.set_index('Entry Time'), on='Entry Time', how='inner')
df

In [ ]:
from sklearn.linear_model import LinearRegression

X = df['Entry Time'].to_numpy().reshape(-1, 1)
Y = df['Waiting Time'].to_numpy().reshape(-1, 1)

linear_regressor = LinearRegression()
linear_regressor.fit(X, Y)
Y_pred = linear_regressor.predict(X)

plt.xlabel('Entry Time')
plt.ylabel('Wait Time')
plt.scatter(X, Y)
plt.plot(X, Y_pred, color='red')
plt.show()

In [ ]:
from sklearn.linear_model import LinearRegression

X = df['Patient in line'].to_numpy().reshape(-1, 1)
Y = df['Waiting Time'].to_numpy().reshape(-1, 1)

linear_regressor = LinearRegression()
linear_regressor.fit(X, Y)
Y_pred = linear_regressor.predict(X)

plt.xlabel('Patient in line')
plt.ylabel('Wait Time')
plt.scatter(X, Y)
plt.plot(X, Y_pred, color='red')
plt.show()

In [ ]:
from sklearn.linear_model import LinearRegression

X = df[['Entry Time', 'Patient in line']].to_numpy()
Y = df['Waiting Time'].to_numpy().reshape(-1, 1)

linear_regressor = LinearRegression()
linear_regressor.fit(X, Y)
Y_pred = linear_regressor.predict(X)

plt.xlabel('Time')
plt.ylabel('Waiting Time')
plt.plot(df['Entry Time'], Y_pred, label='Prediction')
plt.plot(df['Entry Time'], Y, label='Truth')
plt.legend()